In [0]:
%scala
val df = spark.read.format("csv").option("Header","true").load("dbfs:/FileStore/tables/crime.csv")
df.createOrReplaceTempView("Demo")

df: org.apache.spark.sql.DataFrame = [INCIDENT_NUMBER: string, OFFENSE_CODE: string ... 15 more fields]

In [0]:
%sql
SELECT * FROM Demo
LIMIT 5

INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
I182070945,00619,Larceny,LARCENY ALL OTHERS,D14,808,null,2018-09-02 13:00:00,2018,9,Sunday,13,Part One,LINCOLN ST,42.35779134,-71.13937053,"(42.35779134, -71.13937053)"
I182070943,01402,Vandalism,VANDALISM,C11,347,null,2018-08-21 00:00:00,2018,8,Tuesday,0,Part Two,HECLA ST,42.30682138,-71.06030035,"(42.30682138, -71.06030035)"
I182070941,03410,Towed,TOWED MOTOR VEHICLE,D4,151,null,2018-09-03 19:27:00,2018,9,Monday,19,Part Three,CAZENOVE ST,42.34658879,-71.07242943,"(42.34658879, -71.07242943)"
I182070940,03114,Investigate Property,INVESTIGATE PROPERTY,D4,272,null,2018-09-03 21:16:00,2018,9,Monday,21,Part Three,NEWCOMB ST,42.33418175,-71.07866441,"(42.33418175, -71.07866441)"
I182070938,03114,Investigate Property,INVESTIGATE PROPERTY,B3,421,null,2018-09-03 21:05:00,2018,9,Monday,21,Part Three,DELHI ST,42.27536542,-71.09036101,"(42.27536542, -71.09036101)"


In [0]:
%sql
SELECT YEAR, COUNT(OFFENSE_CODE) AS TOTAL_CRIMES FROM Demo
GROUP BY YEAR
ORDER BY YEAR;

YEAR,TOTAL_CRIMES
2015,53388
2016,99114
2017,100886
2018,65685


Output can only be rendered in Databricks

In [0]:
%sql
SELECT CASE WHEN ISNULL(STREET) = false THEN STREET 
            ELSE 'UNKNOWN STREET'
       END AS STREET, COUNT(OFFENSE_CODE) AS CRIME_RECORDED FROM Demo
GROUP BY STREET
ORDER BY CRIME_RECORDED DESC
LIMIT 5;

STREET,CRIME_RECORDED
WASHINGTON ST,14194
UNKNOWN STREET,10871
BLUE HILL AVE,7794
BOYLSTON ST,7221
DORCHESTER AVE,5149


Output can only be rendered in Databricks

In [0]:
%sql
SELECT OFFENSE_DESCRIPTION, DISTRICT, CONCAT (YEAR,'-',MONTH) AS YEAR_MONTH, COUNT(CONCAT (YEAR,'-',MONTH)) AS NUMBER FROM Demo
WHERE OFFENSE_DESCRIPTION = 'MISSING PERSON'
GROUP BY YEAR_MONTH, DISTRICT,OFFENSE_DESCRIPTION
HAVING NUMBER >= 20
ORDER BY NUMBER DESC;

OFFENSE_DESCRIPTION,DISTRICT,YEAR_MONTH,NUMBER
MISSING PERSON,B3,2016-9,39
MISSING PERSON,B3,2016-1,36
MISSING PERSON,B3,2016-8,34
MISSING PERSON,B3,2016-11,33
MISSING PERSON,B3,2016-10,33
MISSING PERSON,B3,2016-4,32
MISSING PERSON,B3,2018-7,32
MISSING PERSON,B2,2017-11,32
MISSING PERSON,B3,2016-7,31
MISSING PERSON,B3,2018-5,30


Output can only be rendered in Databricks

In [0]:
%sql
SELECT 
CASE
WHEN HOUR >= 3 AND HOUR < 6 THEN 'EARLY MORNING'
WHEN HOUR >= 6 AND HOUR < 12 THEN 'MORNING'
WHEN HOUR >= 12 AND HOUR < 16 THEN 'AFTER NOON'
WHEN HOUR >= 16 AND HOUR < 22 THEN 'NIGHT'
WHEN HOUR >= 22 AND HOUR <= 23 THEN 'MID NIGHT'
WHEN HOUR >= 0 AND HOUR < 3 THEN 'MID NIGHT'
END AS PART_OF_DAY, COUNT (*) AS CRIMES FROM Demo
GROUP BY PART_OF_DAY;


PART_OF_DAY,CRIMES
EARLY MORNING,11308
MORNING,74454
MID NIGHT,55590
AFTER NOON,69236
NIGHT,108485


Output can only be rendered in Databricks

In [0]:
%sql
SELECT
(SELECT COUNT(*) FROM Demo
WHERE SHOOTING IS NOT null) AS SHOOTING_INCLUDED,
(SELECT COUNT(*) FROM Demo
WHERE SHOOTING IS null) AS SHOOTING_NOT_INCLUDED;

SHOOTING_INCLUDED,SHOOTING_NOT_INCLUDED
1019,318054


Output can only be rendered in Databricks

In [0]:
%sql
SELECT DAY_OF_WEEK, COUNT(*) AS CRIMES FROM Demo
GROUP BY DAY_OF_WEEK
ORDER BY CRIMES;

DAY_OF_WEEK,CRIMES
Sunday,40313
Saturday,44818
Monday,45679
Tuesday,46383
Thursday,46656
Wednesday,46729
Friday,48495


Output can only be rendered in Databricks

In [0]:
%sql
(SELECT CASE WHEN DISTRICT IS NULL THEN 'UNKNOWN DISTRICT' ELSE DISTRICT END AS DISTRICT, COUNT(OFFENSE_CODE) AS TOTAL_CRIMES, 'TOP 5 HIGHEST' AS DESCRIPTION FROM Demo
GROUP BY DISTRICT
ORDER BY TOTAL_CRIMES DESC
LIMIT 5)
UNION ALL
(SELECT CASE WHEN DISTRICT IS NULL THEN 'UNKNOWN DISTRICT' ELSE DISTRICT END AS DISTRICT, COUNT(OFFENSE_CODE) AS TOTAL_CRIMES, 'TOP 5 LOWEST' AS DESCRIPTION FROM Demo
GROUP BY DISTRICT
ORDER BY TOTAL_CRIMES ASC
LIMIT 5);

DISTRICT,TOTAL_CRIMES,DESCRIPTION
B2,49945,TOP 5 HIGHEST
C11,42530,TOP 5 HIGHEST
D4,41915,TOP 5 HIGHEST
A1,35717,TOP 5 HIGHEST
B3,35442,TOP 5 HIGHEST
UNKNOWN DISTRICT,1765,TOP 5 LOWEST
A15,6505,TOP 5 LOWEST
E5,13239,TOP 5 LOWEST
A7,13544,TOP 5 LOWEST
E18,17348,TOP 5 LOWEST


Output can only be rendered in Databricks